In [1]:
import numpy as np
import pandas as pd

In [2]:
# importing data
nyc_taxi = pd.read_parquet("data/yellow_tripdata_2022-03.parquet")
nyc_taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-03-01 00:13:08,2022-03-01 00:24:35,1.0,2.40,1.0,N,90,209,2,10.0,3.0,0.5,0.00,0.0,0.3,13.80,2.5,0.00
1,1,2022-03-01 00:47:52,2022-03-01 01:00:08,1.0,2.20,1.0,N,148,234,2,10.5,3.0,0.5,0.00,0.0,0.3,14.30,2.5,0.00
2,2,2022-03-01 00:02:46,2022-03-01 00:46:43,1.0,19.78,2.0,N,132,249,1,52.0,0.0,0.5,11.06,0.0,0.3,67.61,2.5,1.25
3,2,2022-03-01 00:52:43,2022-03-01 01:03:40,2.0,2.94,1.0,N,211,66,1,11.0,0.5,0.5,4.44,0.0,0.3,19.24,2.5,0.00
4,2,2022-03-01 00:15:35,2022-03-01 00:34:13,1.0,8.57,1.0,N,138,197,1,25.0,0.5,0.5,5.51,0.0,0.3,33.06,0.0,1.25


In [3]:
# drop non essential columns

In [4]:
# store_and_fwd_flag : technical
# congestion_surcharge and airport_fee not part of total amount paid by customer
nyc_taxi.drop(["store_and_fwd_flag", "congestion_surcharge", "airport_fee"],axis=1, inplace=True)
nyc_taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2022-03-01 00:13:08,2022-03-01 00:24:35,1.0,2.40,1.0,90,209,2,10.0,3.0,0.5,0.00,0.0,0.3,13.80
1,1,2022-03-01 00:47:52,2022-03-01 01:00:08,1.0,2.20,1.0,148,234,2,10.5,3.0,0.5,0.00,0.0,0.3,14.30
2,2,2022-03-01 00:02:46,2022-03-01 00:46:43,1.0,19.78,2.0,132,249,1,52.0,0.0,0.5,11.06,0.0,0.3,67.61
3,2,2022-03-01 00:52:43,2022-03-01 01:03:40,2.0,2.94,1.0,211,66,1,11.0,0.5,0.5,4.44,0.0,0.3,19.24
4,2,2022-03-01 00:15:35,2022-03-01 00:34:13,1.0,8.57,1.0,138,197,1,25.0,0.5,0.5,5.51,0.0,0.3,33.06


In [5]:
nyc_taxi.shape

(3627882, 16)

In [6]:
# removing instances with missing values

In [6]:
nyc_taxi.isna().sum()

VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          117814
trip_distance                 0
RatecodeID               117814
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
dtype: int64

In [7]:
nyc_taxi["trip_distance"] = nyc_taxi["trip_distance"] * 1609.34

In [8]:
nyc_taxi["RatecodeID"].value_counts()/len(nyc_taxi)

1.0     0.921587
2.0     0.033114
5.0     0.005875
99.0    0.003550
3.0     0.002237
4.0     0.001156
6.0     0.000007
Name: RatecodeID, dtype: float64

In [9]:
miss_pos = nyc_taxi[nyc_taxi["RatecodeID"].isna()].index

In [10]:
meanrt1 = nyc_taxi[nyc_taxi["RatecodeID"] == 1.0]["total_amount"].mean()
meanrt2 = nyc_taxi[nyc_taxi["RatecodeID"] == 2.0]["total_amount"].mean()
stdrt1 = nyc_taxi[nyc_taxi["RatecodeID"] == 1.0]["total_amount"].std()
stdrt2 = nyc_taxi[nyc_taxi["RatecodeID"] == 2.0]["total_amount"].std()
for pos in miss_pos:
    payment = nyc_taxi["total_amount"][pos]
    if (payment >= (meanrt1 - stdrt1)) | (payment <= meanrt1 + stdrt1):
        nyc_taxi.loc[pos,"RatecodeID"] = 1.0
    elif (payment >= (meanrt2 - stdrt2)) | (payment <= meanrt2 + stdrt2):
        nyc_taxi.loc[pos,"RatecodeID"] = 2.0
    else:
        nyc_taxi.loc[pos,"RatecodeID"] = 1.0

In [11]:
nyc_taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3627882 entries, 0 to 3627881
Data columns (total 16 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   PULocationID           int64         
 7   DOLocationID           int64         
 8   payment_type           int64         
 9   fare_amount            float64       
 10  extra                  float64       
 11  mta_tax                float64       
 12  tip_amount             float64       
 13  tolls_amount           float64       
 14  improvement_surcharge  float64       
 15  total_amount           float64       
dtypes: datetime64[ns](2), float64(10), int64(4)
memory usage: 442.9 MB


In [12]:
nyc_taxi.isna().sum()

VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          117814
trip_distance                 0
RatecodeID                    0
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
dtype: int64

In [12]:
np.round(nyc_taxi["passenger_count"].value_counts()/len(nyc_taxi),2)

1.0    0.72
2.0    0.14
3.0    0.04
0.0    0.02
5.0    0.02
4.0    0.01
6.0    0.01
7.0    0.00
8.0    0.00
9.0    0.00
Name: passenger_count, dtype: float64

In [13]:
nyc_taxi.loc[nyc_taxi[nyc_taxi["passenger_count"].isna()].index, "passenger_count"] = 1.0

In [15]:
nyc_taxi.isna().sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
dtype: int64

In [16]:
# proper data types of attributes

In [17]:
nyc_taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3627882 entries, 0 to 3627881
Data columns (total 16 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   PULocationID           int64         
 7   DOLocationID           int64         
 8   payment_type           int64         
 9   fare_amount            float64       
 10  extra                  float64       
 11  mta_tax                float64       
 12  tip_amount             float64       
 13  tolls_amount           float64       
 14  improvement_surcharge  float64       
 15  total_amount           float64       
dtypes: datetime64[ns](2), float64(10), int64(4)
memory usage: 442.9 MB


In [16]:
nyc_taxi["passenger_count"] = nyc_taxi["passenger_count"].astype("int")
nyc_taxi["RatecodeID"] = nyc_taxi["RatecodeID"].astype("int")

In [37]:
# reducing the size of data

In [17]:
for col in nyc_taxi.columns:
    if nyc_taxi[col].dtype == "int64":
        nyc_taxi[col] = nyc_taxi[col].astype("int16")
    elif nyc_taxi[col].dtype == "float64":
        nyc_taxi[col] = nyc_taxi[col].astype("float64")
    else:
        pass

In [18]:
nyc_taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3627882 entries, 0 to 3627881
Data columns (total 16 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int16         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        int16         
 4   trip_distance          float64       
 5   RatecodeID             int16         
 6   PULocationID           int16         
 7   DOLocationID           int16         
 8   payment_type           int16         
 9   fare_amount            float64       
 10  extra                  float64       
 11  mta_tax                float64       
 12  tip_amount             float64       
 13  tolls_amount           float64       
 14  improvement_surcharge  float64       
 15  total_amount           float64       
dtypes: datetime64[ns](2), float64(8), int16(6)
memory usage: 318.3 MB


In [19]:
nyc_taxi.to_pickle("data/nyc_taxi.pickle")

In [20]:
nyc_taxi = nyc_taxi.sample(frac=1.0)
nyc_taxi = nyc_taxi.reset_index(drop=True)

In [21]:
test_size = int(np.floor(0.25 * nyc_taxi.shape[0]))
test = nyc_taxi[:test_size]
train = nyc_taxi[test_size:] 

In [22]:
test.to_pickle("data/test.pickle")
train.to_pickle("data/train.pickle")

In [23]:
nyc_taxi.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
count,3.627882e+06,3.627882e+06,3.627882e+06,3.627882e+06,3.627882e+06,3.627882e+06,3.627882e+06,3.627882e+06,3.627882e+06,3.627882e+06,3.627882e+06,3.627882e+06,3.627882e+06,3.627882e+06
mean,1.714948e+00,1.376396e+00,9.271874e+03,1.412461e+00,1.649635e+02,1.630534e+02,1.180307e+00,1.393749e+01,1.029540e+00,4.906932e-01,2.649623e+00,4.911327e-01,2.965781e-01,2.059364e+01
std,4.984502e-01,9.596307e-01,9.164573e+05,5.836790e+00,6.503559e+01,6.997796e+01,4.971751e-01,1.320369e+01,1.250998e+00,8.578644e-02,3.071704e+00,1.902065e+00,4.476943e-02,1.653309e+01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-8.950000e+02,-4.500000e+00,-5.000000e-01,-8.432000e+01,-8.300000e+01,-3.000000e-01,-8.953000e+02
25%,1.000000e+00,1.000000e+00,1.770274e+03,1.000000e+00,1.320000e+02,1.130000e+02,1.000000e+00,7.000000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,3.000000e-01,1.184000e+01
50%,2.000000e+00,1.000000e+00,2.945092e+03,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.000000e+01,5.000000e-01,5.000000e-01,2.150000e+00,0.000000e+00,3.000000e-01,1.536000e+01
75%,2.000000e+00,1.000000e+00,5.471756e+03,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,1.550000e+01,2.500000e+00,5.000000e-01,3.260000e+00,0.000000e+00,3.000000e-01,2.182000e+01
max,6.000000e+00,9.000000e+00,4.606894e+08,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,1.777000e+03,1.855000e+01,1.710000e+01,4.660000e+02,2.357000e+02,3.000000e-01,1.783850e+03
